In [ ]:
#loading LIBS
from sklearn import linear_model
from sklearn import ensemble
from sklearn import svm

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

from glob import glob
import os
import cv2

import numpy as np
import pandas as pd
import math
from sklearn.utils import shuffle
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from skimage import measure
#from skimage import filters
import matplotlib.pyplot as plt
import mahotas as mh
%matplotlib inline

In [ ]:
#check Folder
from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

In [ ]:
#Init Files id and path
TRAIN_DATA = "./data/train"
TEST_DATA = "./data/test/"
ADDITIONAL_DATA = "./data/additional"
types = ['Type_1','Type_2','Type_3']
atypes = ['AType_1','AType_2','AType_3']
type_1_files = glob(os.path.join(TRAIN_DATA, types[0], "*.jpg"))
type_1_ids = np.array([s[len(os.path.join(TRAIN_DATA, types[0]))+1:-4] for s in type_1_files])
type_2_files = glob(os.path.join(TRAIN_DATA, types[1], "*.jpg"))
type_2_ids = np.array([s[len(os.path.join(TRAIN_DATA, types[1]))+1:-4] for s in type_2_files])
type_3_files = glob(os.path.join(TRAIN_DATA, types[2], "*.jpg"))
type_3_ids = np.array([s[len(os.path.join(TRAIN_DATA, types[2]))+1:-4] for s in type_3_files])

atype_1_files = glob(os.path.join(ADDITIONAL_DATA, atypes[0], "*.jpg"))
atype_1_ids = np.array([s[len(os.path.join(ADDITIONAL_DATA, atypes[0]))+1:-4] for s in atype_1_files])
atype_2_files = glob(os.path.join(ADDITIONAL_DATA, atypes[1], "*.jpg"))
atype_2_ids = np.array([s[len(os.path.join(ADDITIONAL_DATA, atypes[1]))+1:-4] for s in atype_2_files])
atype_3_files = glob(os.path.join(ADDITIONAL_DATA, atypes[2], "*.jpg"))
atype_3_ids = np.array([s[len(os.path.join(ADDITIONAL_DATA, atypes[2]))+1:-4] for s in atype_3_files])

all_types=0
all_types = types+atypes


test_files = glob(os.path.join(TEST_DATA, "*.jpg"))
test_ids = np.array([s[len(os.path.join(TEST_DATA)):-4] for s in test_files])
all_types.append('Test')

type_ids=[type_1_ids,type_2_ids,type_3_ids,atype_1_ids,atype_2_ids,atype_3_ids,test_ids]

def get_filename(image_id, image_type):
    """
    Method to get image file path from its id and type   
    """
    if image_type == "Type_1" or \
        image_type == "Type_2" or \
        image_type == "Type_3":
        data_path = os.path.join(TRAIN_DATA, image_type)
    elif image_type == "Test":
        data_path = TEST_DATA
    elif image_type == "AType_1" or \
          image_type == "AType_2" or \
          image_type == "AType_3":
        data_path = os.path.join(ADDITIONAL_DATA, image_type)
    else:
        raise Exception("Image type '%s' is not recognized" % image_type)

    ext = 'jpg'
    return os.path.join(data_path, "{}.{}".format(image_id, ext))

def get_image_data(image_id, image_type):
    """
    Method to get image data as np.array specifying image id and type
    """
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


In [ ]:
all_types

In [ ]:
ROI_TRAIN_DATA = "./roi/train/"
ROI_ADDITIONAL_DATA = "./roi/additional/"
ROI_TEST_DATA = "./roi/test/"

roi_type_1_files = glob(os.path.join(ROI_TRAIN_DATA, types[0], "*.jpg"))
roi_type_1_ids = np.array([s[len(os.path.join(ROI_TRAIN_DATA, types[0]))+1:-4] for s in roi_type_1_files])
roi_type_2_files = glob(os.path.join(ROI_TRAIN_DATA, types[1], "*.jpg"))
roi_type_2_ids = np.array([s[len(os.path.join(ROI_TRAIN_DATA, types[1]))+1:-4] for s in roi_type_2_files])
roi_type_3_files = glob(os.path.join(ROI_TRAIN_DATA, types[2], "*.jpg"))
roi_type_3_ids = np.array([s[len(os.path.join(ROI_TRAIN_DATA, types[2]))+1:-4] for s in roi_type_3_files])

roi_atype_1_files = glob(os.path.join(ROI_ADDITIONAL_DATA, atypes[0], "*.jpg"))
roi_atype_1_ids = np.array([s[len(os.path.join(ROI_ADDITIONAL_DATA, atypes[0]))+1:-4] for s in roi_atype_1_files])
roi_atype_2_files = glob(os.path.join(ROI_ADDITIONAL_DATA, atypes[1], "*.jpg"))
roi_atype_2_ids = np.array([s[len(os.path.join(ROI_ADDITIONAL_DATA, atypes[1]))+1:-4] for s in roi_atype_2_files])
roi_atype_3_files = glob(os.path.join(ROI_ADDITIONAL_DATA, atypes[2], "*.jpg"))
roi_atype_3_ids = np.array([s[len(os.path.join(ROI_ADDITIONAL_DATA, atypes[2]))+1:-4] for s in roi_atype_3_files])

roi_test_files = glob(os.path.join(ROI_TEST_DATA,"*.jpg"))
roi_test_ids = np.array([s[len(os.path.join(ROI_TEST_DATA)):-4] for s in roi_test_files])

roi_type_ids=[roi_type_1_ids,roi_type_2_ids,roi_type_3_ids,roi_atype_1_ids,roi_atype_2_ids,roi_atype_3_ids,roi_test_ids]

In [ ]:
def check_zero_size(ids,image_type,i):
    if i == True:
        start = './data/'
    else:
        start = './roi/'
    
    output_list=[]
    print(len(ids))
    for type_ids in ids:
        image_id = type_ids
        path=''
        ext='.jpg'
        additional_path = start+"additional/"
        train_path = start+"train/"
        test_path = start+"test/"
        if image_type == "Type_1" or \
            image_type == "Type_2" or \
            image_type == "Type_3":
            path = train_path+str(image_type)+'/'+str(image_id)+ext
        elif image_type == "AType_1" or \
            image_type == "AType_2" or \
            image_type == "AType_3":
            path = additional_path+str(image_type)+'/'+str(image_id)+ext
        elif image_type == "Test":
            path = test_path+str(image_id)+ext
            
        if os.stat(str(path)).st_size != 0:
            output_list.append(image_id)
    print(len(output_list))
    print("")
    return output_list
            
for i in xrange(0,len(type_ids)):
    for j in xrange(0,len(all_types)):
        if(i==j):
            type_ids[i]=check_zero_size(type_ids[i],all_types[j],True)

In [ ]:
#https://www.kaggle.com/chiszpanski/intel-mobileodt-cervical-cancer-screening/non-cervix-images
def check_no_cervix_image(ids,image_type):
    no_cervix_ATYPE_1 = ['746','2030','4065','4702','4706','6360']
    no_cervix_ATYPE_2 = ['1813','3086']
    print(image_type)
    print(len(ids))
    if image_type =='AType_1':
        ids = filter(lambda v:v not in no_cervix_ATYPE_1,ids)
    elif image_type =='AType_2':
        ids = filter(lambda z:z not in no_cervix_ATYPE_2,ids)
    print(len(ids))
    print("")
    return ids
    
for i in xrange(0,len(type_ids)):
    for j in xrange(0,len(all_types)):
        if(i==j):
            type_ids[i]=check_no_cervix_image(type_ids[i],all_types[j])
    

In [ ]:
type_ids

In [ ]:
def check_counts():
    for i in xrange(0,len(type_ids)):
        print len(type_ids[i])
        print len(roi_type_ids[i])
        print ("")

check_counts()

In [ ]:
#for i in xrange(0,len(type_ids)):
#    for item in type_ids[i]:
#        if item not in roi_type_ids[i]:
#            type_ids[i].remove(item)


for j in xrange(0,len(type_ids)):
    type_ids[j] = filter(lambda v:v not in roi_type_ids[j],type_ids[j])
    
check_counts()

In [ ]:
def maxHist(hist):
    """
    Method to find max area value in histogram 
    https://www.youtube.com/watch?v=g8bSdXCG-lA
    https://www.youtube.com/watch?v=VNbkzsnllsU 
    """
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else: 
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i-position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif(height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()    
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area =  maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea
            



In [ ]:
def maxRect(img):
    """
    Method to find max Rectangle in image (thresholded)
    """
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])



In [ ]:
def cropCircle(img):
    """
    Method to crop image => remove circular frames
    """
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))
    
    
    img = cv2.resize(img, dsize=tile_size)
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    
    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 2)
    
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask,(int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    
    rect = maxRect(ff)
    img_crop = img[min(rect[0],rect[2]):max(rect[0],rect[2]), min(rect[1],rect[3]):max(rect[1],rect[3])]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    
    return img_crop

In [ ]:
def Circle(img):
    """
    Method to crop image => remove circular frames
    """
    #fig, axes = plt.subplots(2, 4,figsize=(25,17))
    #axes[0,0].imshow(img)
    #axes[0,0].set_title('IMG')
    #axes[0,0].axis('off')
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))
    
    img = cv2.resize(img, dsize=tile_size)
    #axes[0,1].imshow(img)
    #axes[0,1].set_title('SIZE')
    #axes[0,1].axis('off')
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    #axes[0,2].imshow(gray)
    #axes[0,2].set_title("GRAY")
    #axes[0,2].axis('off')
    
    
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    #axes[0,3].imshow(thresh)
    #axes[0,3].set_title('THRESH')
    #axes[0,3].axis('off')
    
    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 2)
    #axes[1,0].imshow(ff)
    #axes[1,0].set_title('CONTOURS')
    #axes[1,0].axis('off')
    
    
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask,(int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    #axes[1,1].imshow(ff)
    #axes[1,1].set_title('FILLED')
    #axes[1,1].axis('off')
    
    rect = maxRect(ff)
    img_crop = img[min(rect[0],rect[2]):max(rect[0],rect[2]), min(rect[1],rect[3]):max(rect[1],rect[3])]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    #axes[1,2].imshow(ff)
    #axes[1,2].set_title('RECTANGLE')
    #axes[1,2].axis('off')
    #axes[1,3].imshow(img_crop)
    #axes[1,3].set_title('FINAL')
    #axes[1,3].axis('off')
    return img_crop
    #return img_crop

In [ ]:
new_image = get_image_data(type_1_ids[1],all_types[0])
q=Circle(new_image)
plt.imshow(q)
plt.axis('off')

In [ ]:
#Initial delineation of the cervix
def Ra_space(img, Ra_ratio, a_threshold):
    """
    Method to find ROI (Region of Interest)
    Change colormap to LAB
    A-channel - the higher value = the redder the pixel color
    R - distance of a pixel from the image center. (spartial information => continuous regions)
    """
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in xrange(w):
        for j in xrange(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

In [ ]:
#The image is separated next into two clusters in the 2-D (a-R) feature space;
#Gaussian mixture modeling, initialized by a K-means
def segmentation(img_real,image_id,image_type):
        
    print(image_id, image_type)
        
    #r,g,b = img_real.transpose(2,0,1)
    #r12 = mh.gaussian_filter(r,48.)
    #g12 = mh.gaussian_filter(g,48.)
    #b12 = mh.gaussian_filter(b,48.)
    #img=mh.as_rgb(r12,g12,b12)
            
    
    #plt.figure(figsize=(20,15))
    #plt.subplot(181)
    #plt.axis('off')
    #plt.imshow(img_real)
    img = cropCircle(img_real)
    res = img.copy()

    w = img.shape[0]
    h = img.shape[1]
        
    Ra = Ra_space(img,0.001,150)
    print(Ra.shape)
        
    a_channel = np.reshape(Ra[:,1], (w,h))
    r_channel = np.reshape(Ra[:,0], (w,h))
    third_channel = np.ones_like(a_channel)
    third_channel[third_channel>0] = 255
    result_ra = np.dstack((a_channel, r_channel, third_channel))
    #plt.subplot(182)
    plt.axis('off')
    plt.set_cmap('Reds')
    plt.imshow(r_channel) 

    gmm = GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
    image_array_sample = shuffle(Ra, random_state=0)
    gmm.fit(image_array_sample)
    labels = gmm.predict(Ra)
    labels += 1
    
    # The cluster that has the highest a-mean is selected.
    labels_2D = np.reshape(labels, (w,h))
    
    
    #plt.subplot(183)
    #plt.axis('off')
    #plt.set_cmap('Reds')
    #plt.imshow(labels_2D)
    gg_labels_regions = measure.regionprops(labels_2D, intensity_image = a_channel)
    gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
    cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1
    

    mask = np.zeros((w * h,1),'uint8')
    mask[labels==cervix_cluster] = 255
    mask_2D = np.reshape(mask, (w,h))

    cc_labels = measure.label(mask_2D, background=0)
    regions = measure.regionprops(cc_labels)
    areas = [prop.area for prop in regions]

    regions_label = [prop.label for prop in regions]
    largestCC_label = regions_label[areas.index(max(areas))]
    mask_largestCC = np.zeros((w,h),'uint8')
    mask_largestCC[cc_labels==largestCC_label] = 255

    img_masked = img.copy()
    img_masked[mask_largestCC==0] = (0,0,0)
    img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY);
            
    _,thresh_mask = cv2.threshold(img_masked_gray,0,255,0)
    #plt.subplot(184)
    #plt.axis("off")
    #plt.imshow(img_masked)
    
    #plt.subplot(185)
    #plt.axis('off')
    #plt.imshow(thresh_mask)
    kernel = np.ones((11,11), np.uint8)
    thresh_mask = cv2.dilate(thresh_mask, kernel, iterations = 1)
    thresh_mask = cv2.erode(thresh_mask, kernel, iterations = 2)
    _, contours_mask, _ = cv2.findContours(thresh_mask.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    
    main_contour = sorted(contours_mask, key = cv2.contourArea, reverse = True)[0]
                    
    x,y,w,h = cv2.boundingRect(main_contour)
    #roi = img_real[y:y+h,x:x+w]
    roi = img[y:y+h,x:x+w]
        
        
    image = img.copy()
    cv2.rectangle(image,(x,y),(x+w,y+h),255,3)
                            
    #plt.subplot(186)
    #plt.axis('off')
    #plt.imshow(image)
    #plt.subplot(187)
    #plt.axis('off')
    #plt.imshow(roi)
            
           
    roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
    ext='.jpg'
    
    '''
    types = ['Type_1','Type_2','Type_3']
    atypes = ['AType_1','AType_2','AType_3']
    test = 'Test'
    print roi.shape[0], " ", roi.shape[1]
    if image_type in types:
        cv2.imwrite('./roi/train/'+str(image_type)+'/'+str(image_id)+ext,roi)
    elif image_type in atypes:
        cv2.imwrite('./roi/additional/'+str(image_type)+'/'+str(image_id)+ext,roi)
    elif image_type == test:
        cv2.imwrite('./roi/test/'+str(image_id)+ext,roi)
    '''    
for i in xrange(0,1):#len(all_types)-1,len(all_types)):
    for q in xrange(144,145):#len(type_ids[i])):
        img_id = type_ids[i][q]
        img_type = all_types[i]
        img = get_image_data(img_id,img_type)
        segmentation(img,img_id,img_type)


In [ ]:
all_types

In [ ]:
plt.imshow(get_image_data(type_ids[0][2],all_types[0]))

In [ ]:
def get_roi_filename(image_id, image_type):
    """
    Method to get image file path from its id and type   
    """
    if image_type == "Type_1" or \
        image_type == "Type_2" or \
        image_type == "Type_3":
        data_path = os.path.join(ROI_TRAIN_DATA, image_type)
    elif image_type == "Test":
        data_path = ROI_TEST_DATA
    elif image_type == "AType_1" or \
          image_type == "AType_2" or \
          image_type == "AType_3":
        data_path = os.path.join(ROI_ADDITIONAL_DATA, image_type)
    else:
        raise Exception("Image type '%s' is not recognized" % image_type)

    ext = 'jpg'
    return os.path.join(data_path, "{}.{}".format(image_id, ext))

def get_roi_image_data(image_id, image_type):
    """
    Method to get image data as np.array specifying image id and type
    """
    fname = get_roi_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

for i in xrange(0,len(roi_type_ids)):
    for j in xrange(0,len(all_types)):
        if(i==j):
            roi_type_ids[i]=check_zero_size(roi_type_ids[i],all_types[j],False)

In [ ]:
def chist(img):
    img = img // 64
    r,g,b = img.transpose((2,0,1))
    pixels = 1 * r + 4 * b + 16 * g
    hist = np.bincount(pixels.ravel(),minlength = 64)
    hist = hist.astype(float)
    hist = np.log1p(hist)
    return hist

In [ ]:
def make_features(img,img_id,img_type):
    w = img.shape[0]
    h = img.shape[1]
    if w>4 and h>4: 
        sift = cv2.xfeatures2d.SIFT_create()
        surf = cv2.xfeatures2d.SURF_create()
            
        print(img_id, img_type)

        haralick = np.ravel(mh.features.haralick(img))
        color_hist = chist(img)
        pftas=mh.features.pftas(img)
        
        img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        _ , sift_desc = sift.detectAndCompute(img,None)
        _ , surf_desc = surf.detectAndCompute(img,None)
        lbp = mh.features.lbp(img,radius=8,points=6)
        return [img_id,img_type,haralick,color_hist,pftas,sift_desc,surf_desc,lbp]


img_features = []
img_ids = []
img_types = []
img_haralick = []
img_chist = []
img_pftas = []
img_sift = []
img_surf = []
img_lbp = []

for q in xrange(len(roi_type_ids)-1,len(roi_type_ids)):
    for u in xrange(0,len(roi_type_ids[q])):
        print str((float(u)/float(len(roi_type_ids[q])))*100)+'%'
        img = get_roi_image_data(roi_type_ids[q][u],all_types[q])
        name = roi_type_ids[q][u]
        if q == 6:
            img_type = 'Test'
        else:
            h = q % 3
            img_type = all_types[h]
        
        features = make_features(img,name,img_type)
        if features:
            img_ids.append(features[0])
            img_types.append(features[1])
            img_haralick.append(features[2])
            img_chist.append(features[3])
            img_pftas.append(features[4])
            img_sift.append(features[5])
            img_surf.append(features[6])
            img_lbp.append(features[7])

In [ ]:
def check_dims(desc,name):
    size_2 = 0
    size_1 = 0
    size_0 = 0
    none_ = 0

    for i in xrange(0,len(desc)):
        if desc[i] is not None:
            if desc[i].ndim == 0:
                size_0+=1
            elif desc[i].ndim == 1:
                size_1+=1
            elif desc[i].ndim == 2:
                size_2+=1
        else:
            none_+=1
            
    print str(name)+" DIM = 2: "+str(size_2)
    print str(name)+" DIM = 1: "+str(size_1)
    print str(name)+" DIM = 0: "+str(size_0)
    print str(name)+" NONE "+str(none_)
    print "FULL LENGTH: "+str(len(desc))+" | SUM: "+str(size_2+size_1+size_0+none_)+" | WITHOUT NULL "+str(size_2+size_1+size_0)
    print ""


In [ ]:
check_dims(img_pftas,'PFTAS')
check_dims(img_chist,'CHIST')
check_dims(img_haralick,'HARALICK')
check_dims(img_sift,'SIFT')
check_dims(img_surf,'SURF')
check_dims(img_lbp,'LBP')

In [ ]:
def filter_desc(desc):
    result = filter(lambda x:x is not None,desc )
    return result

def fix_desc(desc):
    result = []
    n = len(desc[0][0])
    for i in desc:
        if i is not None:
            result.append(i)
        else:
            result.append([0 for i in xrange(0,n)])        
    return result
    

def concatenate_desc(desc):
    result = np.concatenate(desc)
    return result

def check_desc_len(desc,n):
    filter_desc = filter(lambda x: x is not None,desc )
    result = np.concatenate(filter_desc)
    result = result[::n]
    print len(result)

In [ ]:
check_desc_len(img_sift,1)
check_desc_len(img_surf,1)

In [ ]:
from sklearn.externals import joblib
def bow(desc,n,k,name,check=True):
    filtered_desc = filter_desc(desc)
    
    if check:
        desc = fix_desc(desc)
    
    concatenated = concatenate_desc(filtered_desc)
    #concatenated = np.load('./concatenate_bow/'+str(name)+'.npy')
    concatenated = concatenated[::n]
    print len(concatenated)

    km = KMeans(k)
    
    km.fit(concatenated)
    
    #joblib.dump(km, './models/'+str(name)+'.pkl')
    
    result = []
    for d in desc:
        c = km.predict(d)
        result.append(
            np.array([np.sum(c==ci) for ci in xrange(k)])
        )
    
    result = np.array(result,dtype=float)
    print 'Result len '+str(len(result))
    return result
    
    
    

In [ ]:
from sklearn.externals import joblib
def bow(desc,k,name,check=True):
    desc= desc
    if check:
        desc = fix_desc(desc)

    km = joblib.load('./models/'+str(name)+'.pkl')
    result = []
    for d in desc:
        c = km.predict(d)
        result.append(
            np.array([np.sum(c==ci) for ci in xrange(k)],dtype=float)
        )
    
    return result

In [ ]:
img_surf = bow(img_surf,256,'SURF')

In [ ]:
img_sift = bow(img_sift,256,'SIFT')

In [ ]:
img_surf = bow(img_surf,8,256,'SURF')

In [ ]:
img_sift = bow(img_sift,5,256,'SIFT')

In [ ]:
print len(img_surf),len(img_sift),len(img_haralick),len(img_chist)

In [ ]:
#SAVE FEATURES TO FILES
img_ids = np.array(img_ids)
img_types = np.array(img_types)
img_chist = np.array(img_chist)
img_haralick = np.array(img_haralick)
img_pftas = np.array(img_pftas)
img_sift = np.array(img_sift)
img_surf = np.array(img_surf)
img_lbp = np.array(img_lbp)

def save_feature_to_file(feature,name):
    path = './features/'+str(name)+'.npy'
    np.save(path,feature)
    

save_feature_to_file(img_ids,'id')
save_feature_to_file(img_types,'type')
save_feature_to_file(img_chist,'chist')
save_feature_to_file(img_haralick,'haralick')
save_feature_to_file(img_pftas,'pftas')
save_feature_to_file(img_sift,'sift')
save_feature_to_file(img_surf,'surf')
save_feature_to_file(img_lbp,'lbp')

    

In [ ]:
#OPEN FEATURES FROM FILES
def open_features_from_files(name):
    path = './features/'+str(name)+'.npy'
    x = np.load(path)
    return np.array(x)

img_ids = open_features_from_files('id')
img_types = open_features_from_files('type')
img_chist = open_features_from_files('chist')
img_haralick = open_features_from_files('haralick')
img_pftas = open_features_from_files('pftas')
img_sift = open_features_from_files('sift')
img_surf = open_features_from_files('surf')
img_lbp = open_features_from_files('lbp')


In [ ]:
len(img_ids),len(img_types),len(img_chist[0]),len(img_haralick[0]),len(img_pftas[0]),len(img_lbp[0]),len(img_surf[0]),len(img_sift[0])

In [ ]:
def make_df():
    df = pd.DataFrame()
    df['ID'] = img_ids
    df['TYPE'] = img_types

    def make_columns_from_feature(feature,name,df):
        for i in xrange(0,len(feature[0])):
            column=[]
            for j in xrange(0,len(feature)):
                column.append(feature[j][i])
            
            column_name = str(name)+str(i+1)
            df[column_name] = column
        
        return df
    
    df = make_columns_from_feature(img_haralick,'HARALICK',df)
    df = make_columns_from_feature(img_chist,'CHIST',df)
    df = make_columns_from_feature(img_pftas,'PFTAS',df)
    df = make_columns_from_feature(img_surf,'SURF',df)
    df = make_columns_from_feature(img_sift,'SIFT',df)
    df = make_columns_from_feature(img_lbp,'LBP',df)
    
    return df

df = make_df()

In [ ]:
df.head()

In [ ]:
def encode_category(y):
    le = LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    return y

def logistic_regression():
    clf = linear_model.LogisticRegression()
    return clf

def extra_trees():
    clf = ensemble.ExtraTreesClassifier(n_estimators=100,max_depth=10)
    return clf

def random_forest():
    clf = ensemble.RandomForestClassifier(n_estimators=100,max_depth=10)
    return clf

def suppor_vector_classifier():
    clf = svm.SVC(probability=True)
    return clf

models_dict = {logistic_regression():"Logistic Regression",suppor_vector_classifier():"SVC",random_forest():"Random Forest",extra_trees():"Extra Trees"}
models= models_dict.keys()
names = models_dict.values()



def get_metrics(y_pred,y_test,clf_probs):
    accuracy = accuracy_score(y_test,y_pred)
    ll = log_loss(y_test,clf_probs)
    return accuracy,ll



In [ ]:
LDA = joblib.load('./models/LDA.pkl')
test_x = LDA.transform(df[df.columns[2:]])
len(test_x)

In [ ]:
ETc = joblib.load('./models/ETc.pkl') 
y_pred=ETc.predict(test_x)
y_proba=ETc.predict_proba(test_x)
y_pred, y_proba

In [ ]:
len(y_proba)

In [ ]:
type1 = []
type2 = []
type3 = []
#for i in xrange(0,len(img_ids)):
#    result = str(img_ids[i])+'.jpg'
#    img_ids_new.append(result)

for h in xrange(0,len(y_proba)):
    for p in xrange(0,len(y_proba[h])):
        if p==0:
            type1.append(y_proba[h][p])
        elif p==1:
            type2.append(y_proba[h][p])
        elif p==2:
            type3.append(y_proba[h][p])

In [ ]:
report_df = pd.DataFrame()
report_df['image_name']=img_ids
report_df['Type_1']=type1
report_df['Type_2']=type2
report_df['Type_3']=type3

In [ ]:
report_df['image_name'] = report_df['image_name'].astype(int)
report_df.dtypes

In [ ]:
report_df=report_df.sort_values(by=['image_name'], ascending=[True])
report_df.reset_index(drop=True)

In [ ]:
report_df['image_name'] = map(lambda x: str(x)+'.jpg',report_df['image_name'])
report_df.head()

In [ ]:
report_df.dtypes

In [ ]:
report_df.to_csv('submission2.csv',index=False)

In [ ]:
df = df.sample(frac=1)
df[df.columns[1]]=encode_category(df[df.columns[1]])

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=1)
X_trans1 = lda.fit_transform(df[df.columns[2:]],df[df.columns[1]])

lda = LinearDiscriminantAnalysis(n_components=2)
X_trans2 = lda.fit_transform(df[df.columns[2:]],df[df.columns[1]])

In [ ]:
x=[]

for i in xrange(0,len(X_trans1)):
    x.append(X_trans1[i])
    
z=df[df.columns[1]]
plt.figure(figsize=(12,8))
plt.plot(x,z,'o',alpha=.5,c='r',ms=10.0)
plt.title('LDA [Dimension=1]')
plt.ylabel('Class')
plt.xlabel('Coordinates')
plt.yticks([0,1,2])
plt.xlim(-6,5)


In [ ]:
x=[]
y=[]

for i in xrange(0,len(X_trans2)):
    x.append(X_trans2[i][0])
    y.append(X_trans2[i][1])
    

z=df[df.columns[1]]
plt.figure(figsize=(12,8))
plt.yticks([0,1,2])
plt.ylim(-0.1,2.1)
plt.xlim(0,8300)
plt.title('LDA [Dimension=2]')
plt.ylabel('Class')
plt.xlabel('Coordinates')
plt.plot(x,y,z,'.',alpha=.5,c='b',ms=15)

In [ ]:
def check_model(model,x,y):
    
    clf = model

    y = encode_category(y)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33,random_state=100)
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    
    clf_probs = clf.predict_proba(x_test)
    
    result_list = [y_pred,y_test,clf_probs]

    return result_list

'''
x = df[df.columns[2:]]
x=X_trans2
y = df[df.columns[1]]
for i in xrange(0,len(models)):
    models_result = []
    result = check_model(models[i],x,y)
    acc,ll=get_metrics(result[0],result[1],result[2])
    
    print models[i]
    print ""
    print "logloss: "+str(ll)
    print ""
    print "==================="
'''


In [ ]:
'''
TEST/TRAIN

SVC 0.541713014461 0.989315787144
Extra Trees 0.626622172785 0.825656416573
Logistic Regression 0.555802743789 1.08780614121
Random Forest 0.625880608083 0.830826543663



FULL 

Logistic Regression 0.732958022274 0.67936808814
Extra Trees 1.0 2.10942374679e-15
SVC 1.0 0.0560539662167
Random Forest 1.0 0.209240477966 
'''

In [ ]:
cw = []
for i in [None,'balanced']:
    print "class weight = "+str(i)
    print "==========================="
    clf = linear_model.LogisticRegression(C=1,class_weight=i)
    y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=3)
    proba  = cross_val_predict(clf,X_trans2,df[df.columns[1]],method='predict_proba',cv=3)
    acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
    print "cv:"
    print ll
    print ""
    cw.append(ll)

In [ ]:
x = [0,5,10,13,15,20]
plt.figure(figsize=(12,10))
plt.title('Class Weight optimization')
plt.ylabel('LogLoss')
plt.xticks([])
plt.plot([0,1],[cw[0]+0.005,cw[0]+0.005],label='None',lw=4)
plt.plot([0,1],[cw[1],cw[1]], lw=4 ,label='Balanced')
plt.legend(loc="best")

In [ ]:
q = map(lambda x:x+0.005,c_ll)

In [ ]:
c = [0.0001,0.001,0.01,0.1,1]

plt.figure(figsize=(12,10))
plt.title('C optimization')
plt.ylabel('LogLoss')
plt.xlabel('C')
plt.xlim([-0.01,0.85])
plt.xticks([0,0.2,0.4,0.6,0.8],c)
plt.plot(c,q,label='None',lw=4)

In [ ]:
c_ll,

In [ ]:
c = [0.0001,0.001,0.01,0.1,1]
c_ll =[]
for i in c:
    print "C = "+str(i)
    print "==========================="
    clf = linear_model.LogisticRegression(C=i,class_weight=None)
    y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=3)
    proba  = cross_val_predict(clf,X_trans2,df[df.columns[1]],method='predict_proba',cv=3)
    acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
    print "Test/Train prediction result:"
    print ll
    c_ll.append(ll)
    print "==========================="

In [ ]:
clf = linear_model.LogisticRegression(C=1,class_weight='balanced',max_iter=100)
res = check_model(clf,X_trans2,df[df.columns[1]])
_ , ll = get_metrics(result[0],result[1],result[2])
clf.fit(X_trans2,df[df.columns[1]])
print ll

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
              "class_weight":[None,'balanced'],
              "C":[0.0001,0.001,0.01,0.1,1]
             }
clf = linear_model.LogisticRegression(random_state=100)
grid_search = GridSearchCV(clf, param_grid=param_grid,scoring="neg_log_loss")
grid_search.fit(X_trans2, df[df.columns[1]])
print grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
clf = ensemble.ExtraTreesClassifier(n_estimators=100,max_depth=10, random_state=100)
#clf = linear_model.LogisticRegression()
#clf = svm.SVC(probability=True)
result = check_model(clf,X_trans2,df[df.columns[1]])
acc,ll=get_metrics(result[0],result[1],result[2])
    
print ""
print "logloss: "+str(ll)
print ""
print confusion_matrix(result[0],result[1])
print "==================="

In [ ]:
indexes = [None,5,10,13,15,20]
ll_cv =[]
ll_ncv =[]

for i in indexes:
    print "depth = "+str(i)
    print "==========================="
    clf = ensemble.ExtraTreesClassifier(n_estimators=100,max_depth=(i),random_state=100)
    y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=3)
    proba  = cross_val_predict(clf,X_trans2,df[df.columns[1]],method='predict_proba',cv=3)
    acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
    print "Test/Train prediction result:"
    print ll
    ll_cv.append(ll)
    print "==========================="

    print "Full prediction result:"
    clf.fit(X_trans2,df[df.columns[1]])
    y_pred = clf.predict(X_trans2)
    proba = clf.predict_proba(X_trans2)
    acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
    print ll
    ll_ncv.append(ll)
    print "==========================="
    print  " "
    print  " "
    print  " "

In [ ]:
x = [0,5,10,13,15,20]
plt.figure(figsize=(12,10))
plt.title('Depth optimization')
plt.ylabel('LogLoss')
plt.xlabel('Depth')
plt.plot(x,ll_ncv,lw=4,label = "Train curve")
plt.plot(x,ll_cv,lw=4, label = "CV curve")
plt.plot([10,10],[0,1.4], lw=2 ,label='Selected')
plt.legend(loc="lower right")

In [ ]:
indexes = range(0,201)

accuracys = []
log_losses= []
for i in indexes[10::10]:
    clf=ensemble.ExtraTreesClassifier(n_estimators=i,random_state=100,max_depth=10)
    df[df.columns[1]]=encode_category(df[df.columns[1]])
    y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=3)
    proba  = cross_val_predict(clf,X_trans2,df[df.columns[1]],method='predict_proba',cv=3)
    acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
    accuracys.append(acc)
    log_losses.append(ll)

'''acc,ll=get_metrics(y_pred,df[df.columns[1]],proba)
print ""
print "accuracy: "+str(acc)
print "logloss: "+str(ll)
print ""
print confusion_matrix(y_pred,df[df.columns[1]])
print "==================="'''

In [ ]:
x = indexes[10::10]

plt.figure(figsize=(11,6))
plt.plot(x,log_losses, lw=4, label='CV Curve')
plt.plot([100,100],[0.7,0.8],lw=4,label='Selected')
plt.yticks([0.7,0.8])
plt.title('N_estimators optimization')
plt.ylabel('LogLoss')
plt.xlabel('N_estimators')
plt.legend(loc='lower right')


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
              "max_features":[None,'sqrt','log2'],
              "criterion": ["gini", "entropy"],
              "bootstrap": [True, False],
              "min_samples_leaf":[1,2,3,4,5]
             }
clf = ensemble.ExtraTreesClassifier(max_features=None,n_estimators=100,
                                    random_state=100,max_depth=10)
grid_search = GridSearchCV(clf, param_grid=param_grid,scoring="neg_log_loss")
grid_search.fit(X_trans2, df[df.columns[1]])

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
cw = []
for i in [None,'balanced']:
    print "class weight = "+str(i)
    print "==========================="
    clf = ensemble.ExtraTreesClassifier(max_features=None, n_estimators=100, random_state=100, max_depth=10,
                                       criterion="gini",min_samples_leaf=3,class_weight=i)
    y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=3)
    proba  = cross_val_predict(clf,X_trans2,df[df.columns[1]],method='predict_proba',cv=3)
    acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
    print "cv:"
    print ll
    print ""
    cw.append(ll)

In [ ]:
clf = linear_model.LogisticRegression(C=1)#,class_weight='balanced')

y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=4)
proba = cross_val_predict(clf,X_trans2,df[df.columns[1]],method="predict_proba",cv=4)

acc, ll = get_metrics(y_pred,df[df.columns[1]],proba)
print acc,ll
print ""
print confusion_matrix(y_pred,df[df.columns[1]])
print ""
target_names = ['Type_1','Type_2','Type_3']
print classification_report(y_pred,df[df.columns[1]],target_names=target_names)

In [ ]:
clf = ensemble.ExtraTreesClassifier(n_estimators=100,random_state=100,bootstrap=False,
                                   criterion = 'gini',max_depth=10,max_features=None,
                                    min_samples_leaf = 2)

y_pred = cross_val_predict(clf,X_trans2,df[df.columns[1]],cv=4)
proba  = cross_val_predict(clf,X_trans2,df[df.columns[1]],method='predict_proba',cv=4)

#clf.fit(X_trans2,df[df.columns[1]])
#y_pred = clf.predict(X_trans2)
#proba = clf.predict_proba(X_trans2)

acc,ll=get_metrics(y_pred,df[df.columns[1]],proba)
print ""
print "accuracy: "+str(acc)
print "logloss: "+str(ll)
print ""
print confusion_matrix(y_pred,df[df.columns[1]])
print "==================="

target_names = ['Type_1','Type_2','Type_3']
print classification_report(y_pred,df[df.columns[1]],target_names=target_names)

In [ ]:
clf = ensemble.ExtraTreesClassifier(n_estimators=100,random_state=100,bootstrap=True,
                                   criterion = 'gini',max_depth=10,oob_score=True,
                                   max_features=None,min_samples_leaf = 2)
clf.fit(X_trans2,df[df.columns[1]])
odf = clf.oob_decision_function_
y_pred = clf.predict(X_trans2)

In [ ]:
odf,y_pred

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from scipy import interp

# Binarize the output
#y = label_binarize(y_pred, classes=[0, 1, 2])
y = label_binarize(df[df.columns[1]], classes=[0, 1, 2])
n_classes = y.shape[1]

In [ ]:
def calculate_roc_auc(y_test,y_proba, n_classes):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    y_score = []
        
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i],y_proba[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    
    return fpr,tpr,roc_auc

fpr,tpr,roc_auc = calculate_roc_auc(y,odf,n_classes)

roc_auc

In [ ]:
plt.figure(figsize=(16,10))
lw = 5
i=0
plt.plot(fpr[i], tpr[i],linestyle='-',
         lw=lw, label='ROC curve, class '+str(i+1)+' (area = %0.3f)' % roc_auc[i])
plt.plot(fpr[i+1], tpr[i+1],linestyle='-',
         lw=lw, label='ROC curve, class '+str(i+2)+' (area = %0.3f)' % roc_auc[(i+1)])
plt.plot(fpr[i+2], tpr[i+2], color='red',linestyle='-',
         lw=lw, label='ROC curve, class '+str(i+3)+' (area = %0.3f)' % roc_auc[(i+2)])
plt.plot([0.182574,1],[0, 1-0.182574], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.01])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC CURVES')
plt.legend(loc="lower right")

In [ ]:
from sklearn.externals import joblib

clf = ensemble.ExtraTreesClassifier(n_estimators=100,random_state=100,bootstrap=False,
                                   criterion = 'gini',max_depth=10,
                                   max_features=None,min_samples_leaf = 3)
clf.fit(X_trans2,df[df.columns[1]])


In [ ]:

joblib.dump(clf, './models/ETc.pkl')
joblib.dump(lda,'./models/LDA.pkl')

In [ ]:
ETc = joblib.load('./models/ETc.pkl') 
y_pred=ETc.predict(X_trans2)
y_proba=ETc.predict_proba(X_trans2)

acc,ll = get_metrics(y_pred,df[df.columns[1]],y_proba)
print acc,ll

In [ ]:
LDA = joblib.load('./models/LDA.pkl')
x_trans = LDA.transform(df[df.columns[2:]])
x_trans

In [ ]:
import seaborn as sns
sns.barplot([1,2,3,4,5],[0,0,0,2,1])